<a href="https://colab.research.google.com/github/sairakshith2004/fundingchatbot.ipynb/blob/main/Copy_of_fundingchatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fuzzywuzzy[speedup]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.5 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving funders_apply_steps.txt to funders_apply_steps (1).txt
Saving funders_info.txt to funders_info (1).txt


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz, process
nltk.download('punkt')
nltk.download('stopwords')
funders = {
    "technology": ["Department of Science and Technology (DST)", "NASSCOM Foundation", "Indian Angel Network", "Startup India Seed Fund", "Tech India Ventures"],
    "environment": ["Tata Trusts (Sustainability)", "ONGC CSR for Environmental Projects","Sustainable India Finance Facility", "India Climate Collaborative (ICC)", "Green India Mission"],
    "healthcare": ["Indian Council of Medical Research (ICMR)", "Biotechnology Industry Research Assistance Council (BIRAC)", "Tata Trusts (Healthcare)", "HCL Foundation (Healthcare Projects)", "Bill & Melinda Gates Foundation (India)"],
    "education": ["Central Square Foundation", "Azim Premji Foundation", "Infosys Foundation", "Teach for India", "Ratan Tata Trust (Education)"],
    "agriculture": ["National Bank for Agriculture and Rural Development (NABARD)", "Agri Innovation Fund", "AgriTech India", "Indian Council of Agricultural Research (ICAR)", "Tata Chemicals Society for Rural Development"]
}
category_keywords = {
    "technology": ["technology", "tech", "software", "hardware", "artificial intelligence", "data science"],
    "environment": ["environment", "climate", "sustainability", "green", "renewable", "ecology"],
    "healthcare": ["health", "medicine", "medical", "pharmaceutical", "biotech", "biotechnology", "hospital", "healthcare"],
    "education": ["education", "school", "learning", "students", "teaching", "university", "edtech"],
    "agriculture": ["agriculture", "farming", "food", "crops", "farm", "rural", "agribusiness", "agrifood", "agritech"]
}

def preprocess_input(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return filtered_tokens

def match_funders_nltk(tokens):
    matched_funders = []
    for category, keywords in category_keywords.items():
        if any(token in keywords for token in tokens):
            matched_funders.append(funders[category])
    if not matched_funders:
        all_keywords = [keyword for sublist in category_keywords.values() for keyword in sublist]
        suggested_keywords = process.extract(" ".join(tokens), all_keywords, scorer=fuzz.partial_ratio)
        if suggested_keywords:
            print("\nIt looks like you're describing something similar to these areas:")
            for suggestion in suggested_keywords[:3]:
                print(f"- {suggestion[0]}")
            print("Try adjusting your project description based on these terms for better matches.")
        return None
    return [item for sublist in matched_funders for item in sublist] if matched_funders else None

def get_institution_info(institution_name):
    try:
        with open('/content/drive/MyDrive/funders_info.txt', 'r') as file:
            lines = file.readlines()
            info = []
            start_reading = False
            for line in lines:
                if institution_name.lower() in line.lower():
                    start_reading = True
                    info.append(line.strip())  # Add the institution name
                elif start_reading:
                    if line.strip() == "":  # Stop reading when encountering an empty line (end of section)
                        break
                    info.append(line.strip())  # Add the institution's details
            return "\n".join(info) if info else "No additional information found."
    except FileNotFoundError:
        return "Sorry, I couldn't find the detailed information file."

def get_application_steps(institution_name):
    try:
        with open('/content/drive/MyDrive/funders_apply_steps.txt', 'r') as file:
            lines = file.readlines()
            steps = []
            start_reading = False
            for line in lines:
                if institution_name.lower() in line.lower():
                    start_reading = True
                    steps.append(line.strip())  # Add the institution name
                elif start_reading:
                    if line.strip() == "":
                        break
                    steps.append(line.strip())  # Add the application steps
            return "\n".join(steps) if steps else "No application steps found."
    except FileNotFoundError:
        return "Sorry, I couldn't find the application steps file."

def chatbot_greeting():
    print("Hello! I'm your project funding assistant. I'm here to help you find funding institutions in India based on your project ideas.")
    print("Just describe your project briefly, and I’ll recommend potential funders based on the area it falls under (e.g., technology, environment, healthcare, education, agriculture).")

def funding_chatbot_nltk():
    chatbot_greeting()
    while True:
        project_idea = input("\nTell me about your project idea: ")
        if project_idea.lower() == "exit":
            print("Goodbye! I hope you found the information helpful. Feel free to return anytime.")
            break
        tokens = preprocess_input(project_idea)
        matching_funders = match_funders_nltk(tokens)
        if matching_funders:
            print("\nBased on your project, here are some potential funders in India:")
            for funder in matching_funders:
                print(f"- {funder}")
            user_response = input("\nWould you like to know more about any of these institutions? If yes, please type the institution name (or type 'no'): ").strip()
            if user_response.lower() != 'no':
                institution_info = get_institution_info(user_response)
                print(f"\nHere is the detailed information about {user_response}:\n{institution_info}")
                apply_response = input("\nWould you like to know how to apply for this institution? (yes/no): ").strip().lower()
                if apply_response == 'yes':
                    application_steps = get_application_steps(user_response)
                    print(f"\nHere are the steps to apply for {user_response}:\n{application_steps}")
        else:
            print("Sorry, I couldn't find any funders that match your project area. Maybe try describing it differently?")
        continue_chat = input("\nWould you like to search for more funders? (yes/no): ").strip().lower()

        if continue_chat != 'yes':
            print("Goodbye! Best of luck with your project!")
            break
funding_chatbot_nltk()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Hello! I'm your project funding assistant. I'm here to help you find funding institutions in India based on your project ideas.
Just describe your project briefly, and I’ll recommend potential funders based on the area it falls under (e.g., technology, environment, healthcare, education, agriculture).

Tell me about your project idea: agriculture

Based on your project, here are some potential funders in India:
- National Bank for Agriculture and Rural Development (NABARD)
- Agri Innovation Fund
- AgriTech India
- Indian Council of Agricultural Research (ICAR)
- Tata Chemicals Society for Rural Development

Would you like to know more about any of these institutions? If yes, please type the institution name (or type 'no'): no

Would you like to search for more funders? (yes/no): no
Goodbye! Best of luck with your project!
